In [68]:
from keras.preprocessing.image import load_img, img_to_array
import matplotlib.pyplot as plt
import os
import numpy as np

pic_size = 48                         


base_path = "/Users/pratiksha/Documents/Pratiksha/GitHub/Face-expression-recognition-with-Deep-Learning/images"

# Function to preprocess images
def preprocess_image(img_path, target_size=(48, 48)):
    """Load and preprocess an image."""
    img = load_img(img_path, target_size=target_size, color_mode="grayscale")  # Grayscale for custom CNNs
    img_array = img_to_array(img)  # Convert to NumPy array
    img_array /= 255.0  # Normalize pixel values
    return img_array

# Function to load and preprocess dataset
def load_dataset(base_path, subset='train'):
    """Load and preprocess the dataset."""
    images = []
    labels = []
    label_map = {'angry': 0, 'disgust': 1, 'fear': 2, 'happy': 3, 'neutral': 4, 'sad': 5, 'surprise': 6}

    subset_path = os.path.join(base_path, subset)
    for expression in os.listdir(subset_path):
        expression_path = os.path.join(subset_path, expression)
        if not os.path.isdir(expression_path):
            continue  

        for img_name in os.listdir(expression_path):
            img_path = os.path.join(expression_path, img_name)
            try:
                img_array = preprocess_image(img_path)
                images.append(img_array)
                labels.append(label_map[expression])  
            except Exception as e:
                print(f"Error loading image {img_name}: {e}")

    return np.array(images), np.array(labels)



# Function to visualize sample images
def visualize_samples(base_path, subset='train'):
    """Visualize sample images for each expression."""
    plt.figure(figsize=(12, 20))           
    counter = 0                            
    subset_path = os.path.join(base_path, subset)
    
    for expression in os.listdir(subset_path):                      
        expression_path = os.path.join(subset_path, expression)
        if not os.path.isdir(expression_path):
            continue                                                
        
        for i, img_name in enumerate(os.listdir(expression_path)[:5]):  
            counter += 1                                               
            plt.subplot(7, 5, counter)                                  
            img_path = os.path.join(expression_path, img_name)
            img_array = preprocess_image(img_path)
            plt.imshow(img_array, cmap="gray")                         
            plt.xlabel(expression)
    
    plt.tight_layout()
    plt.show()


# Function to count images per expression
def count_images_per_expression(base_path, subset='train'):
    """Count and display the number of images per expression."""
    subset_path = os.path.join(base_path, subset)
    for expression in os.listdir(subset_path):
        expression_path = os.path.join(subset_path, expression)
        if not os.path.isdir(expression_path):
            continue
        
        num_images = len(os.listdir(expression_path))
        print(f"{num_images} {expression} images")


In [69]:
count_images_per_expression(base_path, subset='train')
##visualize_samples(base_path, subset='train')

7164 happy images
4938 sad images
4103 fear images
3205 surprise images
4982 neutral images
3993 angry images
436 disgust images


In [70]:
count_images_per_expression(base_path, subset='validation')
#visualize_samples(base_path, subset='validation')

1825 happy images
1139 sad images
1018 fear images
797 surprise images
1216 neutral images
960 angry images
111 disgust images


In [71]:
#### Error handling:---- don't run now
#for img_name in os.listdir(expression_path):
   # img_path = os.path.join(expression_path, img_name)
   # try:
    #    img_array = preprocess_image(img_path)
     #   images.append(img_array)
     #   labels.append(label_map[expression])
   # except Exception as e:
   #     print(f"Error loading image {img_name}: {e}")


In [72]:
train_images, train_labels = load_dataset(base_path, subset='train')
test_images, test_labels = load_dataset(base_path, subset='validation')

print("Training images shape:", train_images.shape)
print("Training labels shape:", train_labels.shape)
print("Test images shape:", test_images.shape)
print("Test labels shape:", test_labels.shape)


Training images shape: (28821, 48, 48, 1)
Training labels shape: (28821,)
Test images shape: (7066, 48, 48, 1)
Test labels shape: (7066,)


-----------------------------------------------------------------------------------

In [82]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.utils.class_weight import compute_class_weight
import numpy as np

# Size of the image: 48x48 pixels
pic_size = 48                                                       


# Input path for the images
base_path = "/Users/pratiksha/Documents/Pratiksha/GitHub/Face-expression-recognition-with-Deep-Learning/images"

batch_size = 64   

train_datagen = ImageDataGenerator(
    rescale=1.0/255,             
    rotation_range=10,            
    width_shift_range=0.1,        
    height_shift_range=0.1,               
    zoom_range=0.1,               
    horizontal_flip=True,        
    fill_mode='nearest'           
)

      
validation_datagen = ImageDataGenerator(rescale=1.0/255)

In [83]:

train_generator = train_datagen.flow_from_directory(base_path + "/train",           
                                                    target_size=(pic_size,pic_size), 
                                                    color_mode="grayscale",           
                                                    batch_size=batch_size,            
                                                    class_mode='categorical',         
                                                    shuffle=True)                    
validation_generator = validation_datagen.flow_from_directory(base_path + "/validation",
                                                    target_size=(pic_size,pic_size),
                                                    color_mode="grayscale",
                                                    batch_size=batch_size,
                                                    class_mode='categorical',
                                                    shuffle=False)

Found 28821 images belonging to 7 classes.
Found 7066 images belonging to 7 classes.


In [85]:
class_labels = train_generator.class_indices
class_weights = compute_class_weight(
    'balanced', 
    classes=np.array(list(class_labels.values())), 
    y=train_generator.classes
)

# Create a dictionary of class weights
class_weight_dict = dict(zip(class_labels.keys(), class_weights))

# Print the class weights
print("Class weights:", class_weight_dict)

Class weights: {'angry': 1.031125898894494, 'disgust': 9.443315858453474, 'fear': 1.0034817729187702, 'happy': 0.5747188322565207, 'neutral': 0.8264322991340254, 'sad': 0.8337962159347335, 'surprise': 1.2846445286382884}


-----------------------------------------------------------------------------------

In [99]:
class_weight_dict = {0: 1.0311, 1: 9.4433, 2: 1.0035, 3: 0.5747, 4: 0.8264, 5: 0.8338, 6: 1.2846}

class_weight_dict = {i: weight for i, weight in enumerate([
    1.031125898894494, 9.443315858453474, 1.0034817729187702,
    0.5747188322565207, 0.8264322991340254, 0.8337962159347335,
    1.2846445286382884
])}


In [86]:
from keras.layers import Dense, Input, Dropout, GlobalAveragePooling2D, Flatten, Conv2D, BatchNormalization, Activation, MaxPooling2D
from keras.models import Model, Sequential
from keras.optimizers import Adam

# Number of labels :
num_labels =  7

In [88]:
from tensorflow.keras.regularizers import l2
from tensorflow.keras.layers import GlobalAveragePooling2D

model = Sequential()
# 1st Convolution layer
model.add(Conv2D(64,(3,3), padding='same', input_shape=(48, 48,1)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))  
model.add(Dropout(0.25)) 

# 2nd Convolution layer
model.add(Conv2D(128, (5, 5), padding='same', kernel_regularizer=l2(0.001)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

# 3rd Convolution layer
model.add(Conv2D(512, (3, 3), padding='same', kernel_regularizer=l2(0.001)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

# 4th Convolution layer
model.add(Conv2D(512, (3, 3), padding='same', kernel_regularizer=l2(0.001)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))


model.add(GlobalAveragePooling2D())
model.add(Dense(128, activation='relu', kernel_regularizer=l2(0.001)))
model.add(Dropout(0.5))
model.add(Dense(7, activation='softmax'))  

In [89]:
opt = Adam(learning_rate=0.0001)                                                                 
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])   

In [ ]:
model.summary()

In [92]:
steps_per_epoch = train_generator.n // train_generator.batch_size
validation_steps = validation_generator.n // validation_generator.batch_size
print(f"Steps per epoch: {steps_per_epoch}, Validation steps: {validation_steps}")

Steps per epoch: 450, Validation steps: 110


In [100]:
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint


checkpoint = ModelCheckpoint(
    "model_weights.keras",
    monitor='val_accuracy',
    verbose=1,
    save_best_only=True,
    mode='max'
)
early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
callbacks=[checkpoint, early_stop]


steps_per_epoch = train_generator.n // train_generator.batch_size
validation_steps = validation_generator.n // validation_generator.batch_size


history = model.fit(
    x=train_generator,
    epochs=10,
    validation_data=validation_generator,
    steps_per_epoch=steps_per_epoch,
    validation_steps=validation_steps,
    class_weight=class_weight_dict,
    callbacks=[checkpoint, early_stop]
)

Epoch 1/10
450/450 ━━━━━━━━━━━━━━━━━━━━ 0s 720ms/step - accuracy: 0.2066 - loss: 2.8983
Epoch 1: val_accuracy improved from -inf to 0.23594, saving model to model_weights.keras
450/450 ━━━━━━━━━━━━━━━━━━━━ 340s 755ms/step - accuracy: 0.2066 - loss: 2.8982 - val_accuracy: 0.2359 - val_loss: 2.7493
Epoch 2/10
  1/450 ━━━━━━━━━━━━━━━━━━━━ 4:52 652ms/step - accuracy: 0.2344 - loss: 2.5357
Epoch 2: val_accuracy did not improve from 0.23594
450/450 ━━━━━━━━━━━━━━━━━━━━ 1s 272us/step - accuracy: 0.2344 - loss: 2.5357 - val_accuracy: 0.0385 - val_loss: 3.3630
Epoch 3/10


2024-11-25 20:28:08.557041: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
2024-11-25 20:28:08.632268: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]


450/450 ━━━━━━━━━━━━━━━━━━━━ 0s 680ms/step - accuracy: 0.2083 - loss: 2.7295
Epoch 3: val_accuracy improved from 0.23594 to 0.28210, saving model to model_weights.keras
450/450 ━━━━━━━━━━━━━━━━━━━━ 322s 715ms/step - accuracy: 0.2083 - loss: 2.7294 - val_accuracy: 0.2821 - val_loss: 2.5490
Epoch 4/10
  1/450 ━━━━━━━━━━━━━━━━━━━━ 5:05 680ms/step - accuracy: 0.2656 - loss: 3.5434
Epoch 4: val_accuracy improved from 0.28210 to 0.57692, saving model to model_weights.keras
450/450 ━━━━━━━━━━━━━━━━━━━━ 1s 435us/step - accuracy: 0.2656 - loss: 3.5434 - val_accuracy: 0.5769 - val_loss: 2.4927
Epoch 5/10


2024-11-25 20:33:31.315817: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
2024-11-25 20:33:31.384472: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]


450/450 ━━━━━━━━━━━━━━━━━━━━ 0s 669ms/step - accuracy: 0.2337 - loss: 2.6058
Epoch 5: val_accuracy did not improve from 0.57692
450/450 ━━━━━━━━━━━━━━━━━━━━ 318s 705ms/step - accuracy: 0.2337 - loss: 2.6057 - val_accuracy: 0.3227 - val_loss: 2.4243
Epoch 6/10
  1/450 ━━━━━━━━━━━━━━━━━━━━ 5:42 762ms/step - accuracy: 0.2344 - loss: 2.4080
Epoch 6: val_accuracy did not improve from 0.57692
450/450 ━━━━━━━━━━━━━━━━━━━━ 1s 273us/step - accuracy: 0.2344 - loss: 2.4080 - val_accuracy: 0.1538 - val_loss: 2.8549
Epoch 7/10


...

450/450 ━━━━━━━━━━━━━━━━━━━━ 0s 7s/step - accuracy: 0.2373 - loss: 2.5060
Epoch 7: val_accuracy did not improve from 0.57692
450/450 ━━━━━━━━━━━━━━━━━━━━ 4254s 9s/step - accuracy: 0.2373 - loss: 2.5058 - val_accuracy: 0.3527 - val_loss: 2.2775
Epoch 8/10
  1/450 ━━━━━━━━━━━━━━━━━━━━ 5:52 784ms/step - accuracy: 0.3594 - loss: 2.6720
Epoch 8: val_accuracy did not improve from 0.57692
450/450 ━━━━━━━━━━━━━━━━━━━━ 1s 336us/step - accuracy: 0.3594 - loss: 2.6720 - val_accuracy: 0.4231 - val_loss: 2.6228
Epoch 9/10


2024-11-25 21:49:44.290276: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
2024-11-25 21:49:44.384173: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]


450/450 ━━━━━━━━━━━━━━━━━━━━ 0s 6s/step - accuracy: 0.2786 - loss: 2.3519
Epoch 9: val_accuracy did not improve from 0.57692
450/450 ━━━━━━━━━━━━━━━━━━━━ 2712s 6s/step - accuracy: 0.2786 - loss: 2.3519 - val_accuracy: 0.3206 - val_loss: 2.2075
Epoch 10/10
  1/450 ━━━━━━━━━━━━━━━━━━━━ 10:37 1s/step - accuracy: 0.2344 - loss: 2.3893

2024-11-25 22:34:58.065756: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
2024-11-25 22:34:58.239370: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]



Epoch 10: val_accuracy did not improve from 0.57692
450/450 ━━━━━━━━━━━━━━━━━━━━ 2s 556us/step - accuracy: 0.2344 - loss: 2.3893 - val_accuracy: 0.3077 - val_loss: 2.7079


In [94]:
print(f"Validation samples: {validation_generator.n}")
print(f"Batch size: {validation_generator.batch_size}")
print(f"Steps per epoch (validation): {len(validation_generator)}")


Validation samples: 7066
Batch size: 64
Steps per epoch (validation): 111


In [96]:
print(class_weight_dict)

{'angry': 1.031125898894494, 'disgust': 9.443315858453474, 'fear': 1.0034817729187702, 'happy': 0.5747188322565207, 'neutral': 0.8264322991340254, 'sad': 0.8337962159347335, 'surprise': 1.2846445286382884}


In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(20,10))

# Plot Loss
plt.subplot(1, 2, 1)
plt.suptitle('Optimizer: Adam', fontsize=10)
plt.ylabel('Loss', fontsize=16)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.legend(loc='upper right')

# Plot Accuracy
plt.subplot(1, 2, 2)
plt.ylabel('Accuracy', fontsize=16)
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.legend(loc='lower right')

plt.show()


In [102]:
# Show the confusion matrix of our predictions

# Compute predictions
predictions = model.predict(validation_generator)
y_pred = [np.argmax(probas) for probas in predictions]
y_test = validation_generator.classes
class_names = list(validation_generator.class_indices.keys())

from sklearn.metrics import confusion_matrix
import itertools

def plot_confusion_matrix(cm, classes, title='Confusion matrix', cmap=plt.cm.Blues):
    cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
    plt.figure(figsize=(10, 10))
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.tight_layout()

# Compute confusion matrix
cnf_matrix = confusion_matrix(y_test, y_pred)
np.set_printoptions(precision=2)

# Plot normalized confusion matrix
plt.figure()
plot_confusion_matrix(cnf_matrix, classes=class_names, title='Normalized confusion matrix')
plt.show()


 31/111 ━━━━━━━━━━━━━━━━━━━━ 38s 475ms/step

KeyboardInterrupt: 

In [ ]:

model.save('/Users/pratiksha/Documents/Pratiksha/Documents/GitHub/GitHub/Face-expression-recognition-with-Deep-Learning/my_model.keras')


he 450/450 in the training log represents the number of steps (batches) that have been completed during the current epoch. It shows the following:

450/450 means that the model has processed all 450 steps (batches) of the current epoch.
The first 450 represents the number of steps that have been completed so far in the epoch.
The second 450 is the total number of steps that the model will go through in this epoch (i.e., the number of batches).
Since the model has reached 450/450, it means the epoch has finished, and all training data has been processed once.

In short, it indicates the current progress of training in terms of the batches completed in the current epoch

- **Batch size** refers to the number of samples the model processes before updating its weights.
- The **steps per epoch** refers to how many batches are needed to complete one full pass through the training dataset.

So, in your case, the training generator is set to:

- **Batch size = 64**: This means each batch contains 64 images.
- **Steps per epoch = 450**: This means that in each epoch, the model will process 450 batches (each containing 64 images). This results in processing a total of \( 450 \times 64 \) images in one epoch.

If we calculate the total number of images processed during one epoch:

\[
\text{Total images per epoch} = 450 \times 64 = 28,800 \text{ images}
\]

This means, in each epoch, the model will go through 28,800 images (divided into 450 batches of 64 images each).

So, when you see `450/450`, it simply means that all 450 batches (of 64 images each) have been processed for that epoch.

You're correct to ask, because the value `450` for the **steps per epoch** is typically determined based on the size of your dataset and the batch size you are using.

If you haven't explicitly set it, then the number `450` is calculated automatically based on the data generator and batch size.

### How `steps_per_epoch` is calculated:
It is determined as:
\[
\text{steps per epoch} = \frac{\text{total number of samples in training set}}{\text{batch size}}
\]

For instance:
- If your training dataset has 28,800 images, and your batch size is 64, then:
\[
\text{steps per epoch} = \frac{28,800}{64} = 450
\]

So, `450` is not something you manually set, but it's the result of this calculation based on your data size and batch size.